## 1. Setup

In [ ]:
pip install pandas psycopg2-binary rapidfuzz rake-nltk

In [2]:
from warnings import filterwarnings
import pandas as pd
import psycopg2 as pg
import re
from rapidfuzz import process, fuzz
from rake_nltk import Rake
import nltk

In [ ]:
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
engine = pg.connect("dbname='clever' user='clever' host='postgres_clever' port='5432' password='clever'")
nltk.download('stopwords')
nltk.download('punkt_tab')

## 2. Load CSV's

### fmcsa_safer_data

In [4]:
df_fmcsa_safer_data = pd.read_sql('select * from public.fmcsa_safer_data', con=engine)

print(df_fmcsa_safer_data.shape)
df_fmcsa_safer_data.head(1)

(394, 53)


,usdot_num,fmcsa_link,user_created,date_created,user_updated,date_updated,entity_type,operating_status,oos_date,legal_name,dba_name,physical_address,phone,mailing_address,mc_num,duns_number,power_units,drivers,mcs_150_form_date,operation_classification,carrier_type,cargo_types,us_vehicle_inspections,us_driver_inspections,us_hazmat_inspections,us_iep_inspections,us_vehicle_out_of_service,us_driver_out_of_service,us_hazmat_out_of_service,us_iep_out_of_service,us_vehicle_out_of_service_pct,us_driver_out_of_service_pct,us_hazmat_out_of_service_pct,us_iep_out_of_service_pct,us_vehicle_natl_avg_oos_pct,us_driver_natl_avg_oos_pct,us_hazmat_natl_avg_oos_pct,us_crashes_fatal,us_crashes_injury,us_crashes_tow,us_crashes_total,canadian_vehicle_inspections,canadian_driver_inspections,canadian_vehicle_out_of_service,canadian_driver_out_of_service,canadian_vehicle_out_of_service_pct,canadian_driver_out_of_service_pct,carrier_safety_rating_rating_date,carrier_safety_rating_review_date,carrier_safety_rating_rating,carrier_safety_rating_type,mileage,mileage_year
0,1032910,https://safer.fmcsa.dot.gov/query.asp?searchtype=ANY&query_type=queryCarrierSnapshot&query_param=USDOT&query_string=1032910,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 20:31:56.995000+00:00,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-17 15:27:51.082000+00:00,CARRIER,AUTHORIZED FOR HHG,None,LONE STAR LOADERS INC,FANTASTIC MOVES,"10610 METRIC #175\n DALLAS, TX 75243",(214) 349-6683,"10610 METRIC #175\n DALLAS, TX 75243",MC-433384,--,7,7.0,2021-10-13,Auth. For Hire,Interstate,"['General Freight', 'Household Goods']",1,9,0,0,0,2,0,0,0.0,0.22,NaN,0.0,0.22,0.07,0.04,0,0,0,0,0,0,0,0,0.0,0.0,None,2009-06-12,None,Non-Ratable,130000.0,2020.0


### fmcsa_complaints

In [5]:
df_fmcsa_complaints = pd.read_sql('select * from public.fmcsa_complaints', con=engine)

print(df_fmcsa_complaints.shape)
df_fmcsa_complaints.head(1)

(474, 7)


,usdot_num,id,user_created,date_created,complaint_category,complaint_year,complaint_count
0,3500694,003e5853-e612-4661-b889-1113a62212ba,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 17:59:52.992000+00:00,Consumer Complaint(Deceptive Business Practices),2021,1


### fmcsa_company_snapshot

In [6]:
df_fmcsa_company_snapshot = pd.read_sql('select * from public.fmcsa_company_snapshot', con=engine)

print(df_fmcsa_company_snapshot.shape)
df_fmcsa_company_snapshot.head(1)

(400, 20)


,usdot_num,fmsca_ai_profile,user_created,date_created,user_updated,date_updated,company_name,mc_num,registered_address,mailing_address,phone_number,fax_number,safety_review_date,num_of_trucks,num_of_tractors,num_of_trailers,hhg_authorization,total_complaints_2021,total_complaints_2022,total_complaints_2023
0,3975699,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=29886143&id2=31716453&f=search%3D5%26ads%3Da%26state%3DTX%26Submit%3DSearch,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 18:26:30.062000+00:00,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 19:05:56.185000+00:00,GOING GOLDENN TRANSPORTATION LLC,1530057.0,"4420 MARK TRAIL WAY\nDALLAS, TX 75232-1047",Same as above,(214)709-8984,None,NaT,1,0,0,False,0,0,0


### fmcsa_companies

In [7]:
df_fmcsa_companies = pd.read_sql('select * from public.fmcsa_companies', con=engine)

print(df_fmcsa_companies.shape)
df_fmcsa_companies.head(1)

(400, 14)


,usdot_num,user_created,date_created,user_updated,date_updated,company_name,company_url,city,state,total_complaints_2021,total_complaints_2022,total_complaints_2023,location,company_type
0,1032910,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 17:39:14.980000+00:00,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16 17:52:11.255000+00:00,FANTASTIC MOVES,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=31048350&id2=30448824&f=search%3D5%26ads%3Da%26state%3DTX%26Submit%3DSearch,DALLAS,TX,0,0,0,"DALLAS, TX",7


### customer_reviews_google

In [8]:
df_customer_reviews_google = pd.read_sql('select * from public.customer_reviews_google', con=engine)

print(df_customer_reviews_google.shape)
df_customer_reviews_google.head(1)

(12039, 27)


,google_id,review_id,name,place_id,location_link,reviews_link,reviews,rating,review_pagination_id,author_link,author_title,author_id,author_image,author_reviews_count,review_text,review_img_url,review_img_urls,review_photo_ids,owner_answer,owner_answer_timestamp,owner_answer_timestamp_datetime_utc,review_link,review_rating,review_timestamp,review_datetime_utc,review_likes,reviews_id
0,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,ChZDSUhNMG9nS0VJQ0FnSUNabHRtNEZREAE,Wildcat Movers - Dallas,ChIJb_75o8gmTIYRi4oDGe4uX58,"https://www.google.com/maps/place/Wildcat+Movers+-+Dallas/@32.9400451,-96.8313573,14z/data=!4m8!1m2!2m1!1sWildcat+Movers+-+Dallas!3m4!1s0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b!8m2!3d32.9400451!4d-96.8313573",https://search.google.com/local/reviews?placeid=ChIJb_75o8gmTIYRi4oDGe4uX58&q=*&authuser=0&hl=en&gl=US,1692.0,4.9,CAESBkVnSUliZw==,https://www.google.com/maps/contrib/118029097041592824924?hl=en-US,Savannah Jackson,118029000000000000000,https://lh3.googleusercontent.com/a/ACg8ocLa63Zw-j85GHj2L7-UWzUzWv2KwCswRihiRCj6xSSu=s120-c-rp-mo-br100,5.0,"We had an amazing experience with Wildcat Movers! Marvin, Marshall, and Rodrigo were so professional and very polite. They made sure everything was where we wanted it to be and were extremely efficient. We had a terrible experience from a different moving company in the past and these guys changed our opinion on movers! We will definitely be using Wildcat Movers in the future and also recommending to all family and friends!",https://lh5.googleusercontent.com/p/AF1QipNLYp-Of9ChXtPBvekfgNbVJl1122ft9cERq3OW,https://lh5.googleusercontent.com/p/AF1QipNLYp-Of9ChXtPBvekfgNbVJl1122ft9cERq3OW,AF1QipNLYp-Of9ChXtPBvekfgNbVJl1122ft9cERq3OW,None,NaT,NaT,https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sChZDSUhNMG9nS0VJQ0FnSUNabHRtNEZREAE!2m1!1s0x0:0x9f5f2eee19038a8b!3m1!1s2@1:CIHM0ogKEICAgICZltm4FQ%7CCgsImu3zpwYQwN_1fA%7C?hl=en-US,5.0,2023-09-09 22:50:02,2023-09-09 22:50:00,NaN,-6962790000000000000


### company_profiles_google_maps

In [9]:
df_company_profiles_google_maps = pd.read_sql('select * from public.company_profiles_google_maps', con=engine)

print(df_company_profiles_google_maps.shape)
df_company_profiles_google_maps.head(1)

(5865, 49)


,google_id,name,site,subtypes,type,category,phone,full_address,borough,street,city,postal_code,state,us_state,country,country_code,latitude,longitude,time_zone,plus_code,area_service,rating,reviews_link,reviews_tags,photo,photos_count,street_view,located_in,working_hours,working_hours_old_format,other_hours,business_status,about,range,posts,logo,description,verified,owner_id,owner_title,owner_link,booking_appointment_link,order_links,location_link,place_id,cid,reviews_id,located_google_id,reviews
0,0x111f53bf849f031:0xb43eff4f62923c5d,Nicolas Boucher P.A,https://www.miamieliteresidences.com/,Real estate agent,Real estate agent,Real estate agent,+1 305-788-8035,"901 S Miami Ave #215, Miami, FL 33130",Downtown Miami,901 S Miami Ave #215,Miami,33130.0,Florida,Florida,United States of America,US,25.765003,-80.193762,America/New_York,None,False,5.0,"https://search.google.com/local/reviews?placeid=ChIJMfBJ-Dv1EQERXTySYk__PrQ&q=luxury+real+estate+agent+near+Miami,+Florida&authuser=0&hl=en&gl=US",None,https://lh5.googleusercontent.com/p/AF1QipPDwyGggMn1JGc7BiwaYYLoFoaA9bI6LBRjIflc=w800-h500-k-no,3.0,https://lh5.googleusercontent.com/p/AF1QipPDwyGggMn1JGc7BiwaYYLoFoaA9bI6LBRjIflc=w1600-h1000-k-no,None,"{""Monday"":""9AM-8PM"",""Tuesday"":""9AM-8PM"",""Wednesday"":""9AM-8PM"",""Thursday"":""9AM-8PM"",""Friday"":""9AM-8PM"",""Saturday"":""9AM-8PM"",""Sunday"":""9AM-8PM""}",Monday:9AM-8PM|Tuesday:9AM-8PM|Wednesday:9AM-8PM|Thursday:9AM-8PM|Friday:9AM-8PM|Saturday:9AM-8PM|Sunday:9AM-8PM,None,OPERATIONAL,"{""Accessibility"":{""Wheelchair accessible entrance"":true,""Wheelchair accessible parking lot"":true}}",None,None,https://lh6.googleusercontent.com/-2uv6Dbn2Rhk/AAAAAAAAAAI/AAAAAAAAAAA/1EMYHsvdDBk/s44-p-k-no-ns-nd/photo.jpg,None,True,113075000000000000000,Nicolas Boucher P.A,https://www.google.com/maps/contrib/113075236995426588165,None,None,"https://www.google.com/maps/place/Nicolas+Boucher+P.A/@25.7650032,-80.1937616,14z/data=!4m8!1m2!2m1!1sNicolas+Boucher+P.A!3m4!1s0x111f53bf849f031:0xb43eff4f62923c5d!8m2!3d25.7650032!4d-80.1937616",ChIJMfBJ-Dv1EQERXTySYk__PrQ,12988100000000000000,-5458640000000000000,None,6.0


## 3. Join Selected df's

### fmcsa df

In [10]:
cols_fmcsa_safer_data = [
    'usdot_num',
    'carrier_type',
    'cargo_types',
    'operating_status',
    'phone',
]

cols_fmcsa_companies = [
    'usdot_num',
    'date_created',
    'company_name',
    'city',
    'state',
    'total_complaints_2023',
]

df_fmcsa = pd.merge(
    df_fmcsa_companies[cols_fmcsa_companies],
    df_fmcsa_safer_data[cols_fmcsa_safer_data],
    on='usdot_num',
    how='inner'
)

print(df_fmcsa.shape)
df_fmcsa.head(1)

(394, 10)


,usdot_num,date_created,company_name,city,state,total_complaints_2023,carrier_type,cargo_types,operating_status,phone
0,1032910,2024-02-16 17:39:14.980000+00:00,FANTASTIC MOVES,DALLAS,TX,0,Interstate,"['General Freight', 'Household Goods']",AUTHORIZED FOR HHG,(214) 349-6683


### google df

In [11]:
cols_customer_reviews_google = [
    'google_id',
    'review_text',
    'owner_answer_timestamp',
    'review_rating',
    'review_timestamp',
    'review_likes',
]

cols_company_profiles_google_maps = [
    'google_id',
    'name',
    'rating',
    'verified',
    'reviews',
    'city',
]

df_google = pd.merge(
    df_customer_reviews_google[cols_customer_reviews_google],
    df_company_profiles_google_maps[cols_company_profiles_google_maps],
    on='google_id',
    how='left'
)

print(df_google.shape)
df_google.head(1)

(12039, 11)


,google_id,review_text,owner_answer_timestamp,review_rating,review_timestamp,review_likes,name,rating,verified,reviews,city
0,0x864c26c8a3f9fe6f:0x9f5f2eee19038a8b,"We had an amazing experience with Wildcat Movers! Marvin, Marshall, and Rodrigo were so professional and very polite. They made sure everything was where we wanted it to be and were extremely efficient. We had a terrible experience from a different moving company in the past and these guys changed our opinion on movers! We will definitely be using Wildcat Movers in the future and also recommending to all family and friends!",NaT,5.0,2023-09-09 22:50:02,NaN,Wildcat Movers - Dallas,4.9,True,NaN,Dallas


### joined df

In [12]:
def clean_name(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)
    return name.strip()

def fuzzy_match(row, column_name, choices, threshold=75):
    value = row[column_name]
    best_match, score, _ = process.extractOne(value, choices, scorer=fuzz.ratio)
    return best_match if score > threshold else None

df_fmcsa['cleaned_name'] = df_fmcsa['company_name'].apply(clean_name)
df_google['cleaned_name'] = df_google['name'].apply(clean_name)

fmcsa_names = df_fmcsa['cleaned_name'].tolist()

df_google['fuzzy_company_name'] = df_google.apply(
    lambda row: fuzzy_match(row, 'cleaned_name', fmcsa_names),
    axis=1
)

df_google[['cleaned_name', 'fuzzy_company_name']].dropna(subset=['fuzzy_company_name']).drop_duplicates().head(10)

,cleaned_name,fuzzy_company_name
1,ab moving,abc moving inc
6,eco movers moving storage,scout moving storage
12,xpress relocations llc,xpress relocations llc
15,wrightway moving company llc,r j moving company llc
20,exodus moving storage llc,ciao moving storage llc
29,phoenix express specialty moving delivery,phoenix express specialty moving delivery llc
543,college hunks hauling junk and moving miami beach,college hunks hauling junk and moving
717,anyway relocation services inc,relocation services inc
839,cant stop moving,cant stop moving llc


In [13]:
df_google['city'] = df_google['city'].str.upper()

df = pd.merge(
    df_google.dropna(subset=['fuzzy_company_name', 'city']),
    df_fmcsa.dropna(subset=['cleaned_name', 'city']),
    left_on=['fuzzy_company_name', 'city'],
    right_on=['cleaned_name', 'city'],
    how='inner'
)

print(df.shape)
df.head(1)

(2802, 23)


,google_id,review_text,owner_answer_timestamp,review_rating,review_timestamp,review_likes,name,rating,verified,reviews,city,cleaned_name_x,fuzzy_company_name,usdot_num,date_created,company_name,state,total_complaints_2023,carrier_type,cargo_types,operating_status,phone,cleaned_name_y
0,0x5490157655555555:0x673e5a56019f797e,Very courteous and efficient. We recommend them and we would use them again. Team leader Christopher from university place was awesome. The person that hired them was my sister Dr. M Robinson.,NaT,5.0,2022-09-09 23:43:44,NaN,Eco Movers Moving & Storage,4.8,True,NaN,SEATTLE,eco movers moving storage,scout moving storage,2030210,2024-02-16 17:40:14.133000+00:00,SCOUT MOVING & STORAGE,WA,0,Interstate,['Household Goods'],AUTHORIZED FOR HHG,(503) 236-6241,scout moving storage


## 4. Filter by City

### Available companies

In [14]:
df[['company_name', 'city']].drop_duplicates()

,company_name,city
0,SCOUT MOVING & STORAGE,SEATTLE
1500,XPRESS RELOCATIONS LLC,MIAMI
1726,R & J MOVING COMPANY LLC,DALLAS
2414,PHOENIX EXPRESS SPECIALTY MOVING & DELIVERY LLC,DALLAS
2789,CAN'T STOP MOVING LLC,SEATTLE


### Select city

In [15]:
df_city = df[df['city']=='SEATTLE']

## 5. Company Rank by Rating

In [16]:
df_city[['company_name', 'rating']].drop_duplicates().sort_values(by=['rating'], ascending=False).head()

,company_name,rating
2789,CAN'T STOP MOVING LLC,5.0
0,SCOUT MOVING & STORAGE,4.8


## 6. Additional Analysis

### Company overview

In [17]:
df_ratings_count = df.rename(columns={'rating': 'rating_count'})[['company_name', 'rating_count']].groupby('company_name').count()

df = pd.merge(
    df,
    df_ratings_count,
    on='company_name',
    how='left',
)

In [18]:
df['overview'] = df.apply(
    lambda row: f"{row['company_name']} has an average rating of {row['rating']} from a total of {row['rating_count']} reviews.",
    axis=1
)

df['overview'].drop_duplicates()

0                               SCOUT MOVING & STORAGE has an average rating of 4.8 from a total of 1500 reviews.
1500                             XPRESS RELOCATIONS LLC has an average rating of 4.5 from a total of 226 reviews.
1726                           R & J MOVING COMPANY LLC has an average rating of 4.7 from a total of 688 reviews.
2414    PHOENIX EXPRESS SPECIALTY MOVING & DELIVERY LLC has an average rating of 4.9 from a total of 375 reviews.
2789                               CAN'T STOP MOVING LLC has an average rating of 5.0 from a total of 13 reviews.
Name: overview, dtype: object

### Company reviews keywords

In [21]:
rake = Rake()

def get_keywords(text):
    if isinstance(text, str) and len(text) > 1:
        rake.extract_keywords_from_text(text)
        ranked_phrases = rake.get_ranked_phrases()
        return [keyword for keyword in ranked_phrases if re.match(r'^[a-zA-Z]+$', keyword)]
    else:
        return None

df['review_keywords'] = df['review_text'].apply(lambda text: get_keywords(text))

df[['review_text', 'review_keywords']].head()

,review_text,review_keywords
0,Very courteous and efficient. We recommend them and we would use them again. Team leader Christopher from university place was awesome. The person that hired them was my sister Dr. M Robinson.,"[robinson, recommend, person, hired, efficient, courteous, awesome]"
1,We've now done 2 moves with them. Everything from the sales to customer service to their crews are nothing short of phenomenal. I would use them again and again!,"[sales, phenomenal, everything, crews]"
2,"Hired Eco Movers to move our office and they did an excellent job! From the beginning of the process of getting a quote, to email reminders the week of all the way to the actual moving day team there were no issues, they were communicative, and made sure all my questions were answered. Also big shout out to the moving team Jeremy and Oscar, they moved things quickly, safely and were extremely thoughtful as to where and how they placed our items in our storage unit. Would definitely recommend them and hope to hire them for our next office move.","[office, move, week, way, safely, quote, questions, process, placed, oscar, items, issues, hope, hire, getting, communicative, beginning, answered]"
3,"Luis, Felix and Sully were great on a fairly challenging move. They were very careful with the household of delicate antiques & the three enormous pieces didn’t faze them one bit!! They kept their positive attitudes even at hour 9, and made the move go smoothly. Awesome team!","[sully, made, luis, kept, household, great, felix, faze, careful]"
4,"We used Eco Movers when we moved into our apartment in Seattle. Can not thank Boris, Alan, and Seg enough for their amazing help! They were careful with our boxes and fragile items. Highly recommend them!! Especially Boris…you were a boss and had great leadership qualities with your team.","[team, seattle, moved, careful, boxes, boss, apartment, alan]"


In [22]:
df_exploded = df.explode('review_keywords')

keyword_counts = df_exploded.groupby(['company_name', 'review_keywords']).size().reset_index(name='count')
keyword_counts = keyword_counts.sort_values(['company_name', 'count'], ascending=[True, False])

top_keywords = keyword_counts.groupby('company_name').head(3)

top_keywords_agg = top_keywords.groupby('company_name')['review_keywords'].apply(list).reset_index()
top_keywords_agg = top_keywords_agg.rename(columns={'review_keywords': 'most_used_keywords'})

top_keywords_agg.head()

,company_name,most_used_keywords
0,CAN'T STOP MOVING LLC,"[careful, used, crew]"
1,PHOENIX EXPRESS SPECIALTY MOVING & DELIVERY LLC,"[move, professional, time]"
2,R & J MOVING COMPANY LLC,"[move, professional, time]"
3,SCOUT MOVING & STORAGE,"[move, professional, boris]"
4,XPRESS RELOCATIONS LLC,"[company, professional, move]"
